In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import re
from openpyxl import Workbook


def read_files(path, sheetname, n_skiprows, n_skip_columns, site_index):
    """
    Params:\n
    path: parth of file in the computer.\n
    n_skiprows: Number of rows to delete in the original file,.\n
    columns_to_convert: Columns to convert the data general type. \n
    n_skipcolumn: Columns to skip in the original file. \n
    endrow = pass 0 to read everything, 1 to count entire
    columns_order: List of columns names in specific order to pass in the engine.\n
    """
    df = pd.read_excel(path, sheet_name = sheetname, skiprows = n_skiprows)

    df = df.iloc[:,n_skip_columns:]

    # define the entiry columns which doesn't have NaN 
    df = df.dropna(subset=[site_index], axis=0)
    """cont = 0
    for i in df.iloc[:,site_col]:
        if pd.isnull(i):
            break # acertar o break
        else:
            cont +=1
    df = df.iloc[:cont, :]
    #df.columns = columns_order"""
    
    # convert intery columns to integer 
    #df[columns_integer_convert] = df[columns_integer_convert].fillna(0)
    #df[columns_integer_convert] = df[columns_integer_convert].astype('int64')

    return df

def lower_str(columns):
    newlist = list(map(lambda x: x.lower(), columns))
    return newlist

def check_df(df, error_msg):
    if df == None or df.empty:
        return f'{error_msg}'
    else:
        return df

def count_duplicates(lista):
    count_dict = {}
    for entry in lista:
        if entry in count_dict.keys():
            count_dict[entry] += 1
        else:
            count_dict[entry] = 1
    
    duplicates = {}
    for k, v in count_dict.items():
        if v > 1:
            duplicates[k] = v
    return pd.DataFrame.from_dict(duplicates, orient='index', columns=['# of Duplicates'])

def defining_df(df, column_range, number_col):
    cont = 0
    for i in df.iloc[:,number_col]:
        if pd.isnull(i):
            break # acertar o break
        else:
            cont +=1
    df = df.iloc[0:cont, :]
    return df

#old version
def check_columns(table, output_columns):
    """
    Check the total of number of missing columns and the missing columns in passed table.\n

    Params:\n
    table: contain the columns to be check.\n
    output_columns: columns structure at the final file. 

    Returns:\n
    Number of missing columns and a list that contains the name os missing columns.
    """
    """"
    countries = ['DE':{'towerDB':[],\
                       'lc': [],\
                       'ta':[]}\,
                 'HU',
                 'IE',
                 'RO',
                 'PT',
                 'ES',
                 'CZ': {'towerDB':["Code (Duplicate)","Site Status","VF - In scope / out of scope (Generalised scoping)","Site in Skylon scope Actual (From Site List Sheet )","Legacy Site Code(Duplicate)","TIMS Site Code","Legacy Site Code","Site Name","Macro Region","Province","Municipality","Inhabitants","Address","Ground Register","Altitude","Latitude","Longitude","Categorization by Inhabitants","Categorization by Transmission Sys","Categorization by Site Type","Categorization by Transmission Sys (subcluster)","Other internal Categorization 1 (Identify ACQ Sites)","Other internal Categorization 2 Energy provider (Eon/ LL)","DAS+Macro","DAS (Yes/ No)","DAS Ownership (Complete/ Partial/ 3rd Party)","Active/ Passive DAS","# of remote units/ radiating points","Type of Structure","Distance highest antenna to ground level","GBT Tower height","POD ID","Energy Consumption LTM (kwh)","Annual Energy cost LTM (Euros)","Infrastructure ready (existing)/ to be ready (new)","Infrastructure to be dismantled by","Radio equipments to be deactivated by","Infrastructure to be shared by","Technology VOD","Fibre / Microwave","Vertical passive structure owner","Room configuration (detailed)","Shelter passive structure ownership","Type of Air Conditioning","Number of cabinets (Full Capacity)","Number of Antenna (Full Capacity)","Number of MW (Full Capacity)","Counterpart","# of Lease Contracts","Current annual lease fees ","Current other fees (Maintenance)","Current other fees","(Average) residual duration - Lease contract","(Average) residual duration - Maintenance contract","(Average) residual duration - Lease & Maintenance both","# of Tenants Agreements","Current Total Annual Hosting Fees","Tenant (name/ID) MNO1 (Česká telekomunikační infrastruktura a.s.)","Annual Fee per Tenant MNO1","Annual Energy Fee MNO1","Annual Maintenance Fee MNO1","Other Services Fee MNO1","Residual duration MNO1 (Years)","Tenant (name/ID) MNO2 (T-Mobile Czech Republic a.s.)","Annual Fee per Tenant MNO2","Annual Energy Fee MNO2","Annual Maintenance Fee MNO2","Other Services Fee MNO2","Residual duration MNO2 (days)","Tenant (name/ID) MNO3","Annual Fee per Tenant MNO3","Annual Energy Fee MNO3","Annual Maintenance Fee MNO3","Other Services Fee MNO3","Residual duration MNO3","# of OTMOs","Annual Fee from OTMOs","Annual Energy Fee from OTMOs","Annual Maintenance Fee OTMOs","Other Services Fee OTMOs","Average residual duration (days)","Check","Strategic Macro Sites","Critical Sites","Case A Core Site","Macro Site - Transmission Hub Site","Macro Site - Transmission Hub Site with/without Shelters","Transmission Sites","Room Configuration","Power Supply","Air Conditioning","Active Sharing Arrangements involving the Operator","VF-CZ Demerger phase","EVO Location [FAR Site ID] ","Billing Trigger date ",\
                 "Strategic_Site_Bucket","Critical Site Bucket","Subsequent_Sharing_Arrangement","First_Active_Sharing_Deployment_Type","First_Active_Sharing_Start_Date","First_Active_Sharing_End_Date","Decommissioned_Site","Wip_Site","Bts_Site","Transfer_Date_Of_Consent_Required_Sites","Sites_As_Metered_Estimated","Date_Of_Equipment_Removal"],\
                       'lc': [],\
                       'ta':['Tenant Agreement ID - NEW', 'Code', 'Site Name', 'Service Type','Contract start date', 'Contract end date', 'Status of contract','Renewal Option', 'Comments', 'Counterpart ID', 'Counterpart','Classification of Tenant', 'Annual amount - billing currency','Billing Currency', 'Annual Amount in CZK', 'Amount in EUR','Terms of Payment', 'Frequency', 'Indexed YES/NO', 'Index','Percentage', 'VAT Subject YES/NO', 'Percentage (VAT)', 'Unique Key','Classification', 'Residual Period in Years', 'Remarks','Count of unique key', 'Count of contracts', 'Scoping classification','DAS sites', 'DAS ownership', '31-Mar-21', 'Update in month','Updated Item', 'Comment', 'Counterpart_extra_1', 'Counterpart_extra_2', 'x','SiteType', '1.028', 'Unique Tenant', 'Unique Tenant Count', 'not_def_1']}\,
                 }
                 'GR']
                 """
    total_received = len(table.columns)
    number_missing_columns = 0
    missing_columns = []
    #Counting of missing columns       
    #if country in contries:      
    for columns in output_columns:
        if columns.lower() not in [labels.lower() for labels in table]:
            number_missing_columns +=1
            missing_columns.append(columns)
    
    return total_received, number_missing_columns, missing_columns

def check_columns_received(df, bill_cols):
    twdb_col = lower_str(list(df.columns))
    col_miss = [i for i in bill_cols if i not in twdb_col]
    """
    for i in bill_cols:
        if i not in twdb_col:
            col_miss.append(i)"""
    df_col_missing = pd.DataFrame(col_miss, columns=['Column(s) Missing'], index=range(len(col_miss)))
    return df_col_missing

def replace_values(df, columns, value=""):
    """
    Está voltando para float
    """
    invalid_values = ['N/A', 'n/a',"0", 0, '-', '_', np.nan,'nan']
    #lista = []

    df.fillna(0, inplace=True)
    #df[column] = df[column].astype('int64')

    for column in columns:
        lista = []
        for index in df[column]:
            #print(f"{column} -> {index}")
            if index in invalid_values:
                lista.append(value)
            else:
                lista.append(index)
        df[column] = lista

    return df
      
def date_parser(df, columns, format, type_dates):
    t_col = type_dates.lower()
    for column in columns:
        if t_col == 'mixed':
            df[column] = [date_obj.strftime(format) if not pd.isnull(date_obj) \
                        and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
            df[column] = df[column].astype(str)
        else:
            df[column] = [date_obj.strftime(format) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
            df[column] = df[column].astype(str)

# Refactorar esse codigo para receber todas as colunas num dic
# Sendo as keys=columns e values= picklist for each column
def check_date_columns(df, df_index, columns, format):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    df_dates = df[columns]
    df_dates['sites'] = df_dates[df_index]
    df_dates = df_dates.set_index('sites')

    df_de = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df_dates[df_index])
    if not df_duplicates.empty:
        df_dates.drop_duplicates(subset=[df_index], inplace=True)
    
    if format == 1:
        date_format = re.compile(r'\d{2}\-\d{2}\-\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                #print(type(df_dates.loc[de_site,de_column]))
                if date_format.match(df_dates.loc[de_site,de_column]) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        df_de.loc[de_site,de_column] = 'Incorret Format or Blank Value'
                    else:
                        df_de.loc[de_site,de_column] = 'Incorret Format or Blank Value'
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            return df_de
        else: 
            print('No one columns with incorrect date format!')
    else:
        date_format = re.compile(r'\d{2}\/\d{2}\/\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                if date_format.match(df_dates.loc[de_site,de_column]) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        df_de.loc[de_site,de_column] = 'Incorret Format or Blank Value'
                    else:
                        df_de.loc[de_site,de_column] = 'Incorret Format or Blank Value'
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            return df_de
        else: 
            print('No one columns with incorrect date format!')

def check_amounts(df_check, df_index, columns, pattern=','):
    """
    Paramns:
    pattern: general (. to decimal), other (, to decimal)
    """

    df = df_check[columns]
    df['sites'] = df[df_index]
    df = df.set_index('sites')

    df_new = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df[df_index])
    if not df_duplicates.empty:
        df.drop_duplicates(subset=[df_index], inplace=True)

    for site in df[df_index]:
        for column in columns[1:]:
            if not str(df.loc[site,column]).__contains__(pattern):
                #print(df.loc[site,column])
                if df.loc[site,df_index] not in df_new.index:
                    df_new.loc[site,df_index] = df.loc[site,df_index]
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'
                else:
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'

    df_new = df_new.dropna(how='all', axis=1).fillna('Ok')       
    if not df_new.empty:
        return df_new
    else: 
        print('No one columns with incorrect Amount format!')
        
def check_picklist(df,df_index,df_cols, picklist_dict):
    """
    Params:
        df:
        df_index: 
        picklist_dict:
        nultiIndex: More than one column to check
    """
    df_picklist = df[df_cols]
    df_picklist['sites'] = df[df_index]
    df_picklist =  df_picklist.set_index('sites')
    
    #df_picklist = replace_values(df_picklist, df_cols, 0)

    df_errors = pd.DataFrame(columns=df_cols)

    for site in df[df_index]:
        for column in picklist_dict.keys(): 
            value = str(df_picklist.loc[site,column])
            col_values = [i.lower() for i in picklist_dict[column]]
            if not value.lower() in col_values or pd.isnull(value):
                if not df_picklist.loc[site,df_index] in df_errors.index:
                    df_errors.loc[site,df_index] = df_picklist.loc[site,df_index]
                    df_errors.loc[site,column] = 'Incorret picklist value or Blank Value'
                else:
                    df_errors.loc[site,column] = 'Incorret picklist value or Blank Value'
    #df = df_errors.dropna()   
    df = df_errors.dropna(how='all', axis=1) 
    
    return df

def check_picklist_3(df,df_index, picklist_dict):
    log = {}
    for column in picklist_dict:
        df_aux = df.copy()
        new = df_aux[column].isin(picklist_dict[column])
        #new = df_aux[column].apply(lambda x: x in picklist_dict[column])
        # Aceita somento os valores que não estão na picklist
        indexes = df.index[new == False].tolist()
        #if len(indexes)>0:
        if column not in log:log[column]=[]
        log[column]=log[column]+indexes
    #print(log.keys())

    newDict ={}
    df1 = pd.DataFrame()
    for key,value in log.items():
        for val in value:  
            ID=df.iloc[val][df_index]
            if ID in newDict:
                newDict[ID].append(key)
            else:
                newDict[ID] = [key]
        
    logs = pd.DataFrame.from_dict(newDict, orient='index')
    return logs

#check on air foi trocado pelo check_picklist
def check_on_air_sites(df, df_index, df_cols):

    df_check = df[df_cols]
    df_check['sites'] = df_check[df_index]
    df_check = df_check.set_index('sites')
    #df_check = replace_values(df_check, df_cols, 0)

    df_errors = pd.DataFrame(columns=df_cols)

    for site in df[df_index]:
        for column in df_cols: 
            if df_check.loc[site,df_index] not in df_errors.index:
                df_errors.loc[site,df_index] = df_check.loc[site,df_index]
                if df_check.loc[site,column] == 0:
                    df_errors.loc[site,column] = 'Incorret picklist value or Blank Value'
            else:
                if df_check.loc[site,column] == 0:
                    df_errors.loc[site,column] = 'Incorret picklist value or Blank Value'
    df = df_errors[df_errors.iloc[:,1:]]
    df = df.dropna(how='all', axis=1).fillna('Ok')       
    #df = df_errors.dropna(how='all', axis=1)   
    return df

def check_new_sites(df_towerdb, tw_index, bts_col, bill_col, msa_list, towerdb_list, uip_list):
    
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    
    #Finding for ALL NEW SITES
    new_site = [str(i) for i in towerdb_list if i not in msa_list]
    new_sites = pd.DataFrame(new_site, columns=['New_Sites'])
    
    #list of new sites out of UIP sheet
    bts_out_uip = [str(i) for i in df_towerdb[df_towerdb[bts_col]=='Yes'][tw_index].sort_values() if i not in uip_list]
    bts_out_uip = pd.DataFrame(bts_out_uip, columns=['Bts_Sites_Out_UIP_File'])

    #create a copy to make index modifications
    df = df_towerdb.copy()

    #New columns with Site Codes
    df['Sites'] = df[tw_index]
    #defining site code to index
    df.set_index('Sites', inplace=True)

    #filtering by new sites
    df = df[df[tw_index].isin(new_site)]

    # Save information os sites with demerged date more than current date
    df[bill_col] = df[bill_col].astype('datetime64[s]')
    df_site_bts = df[(df[bts_col]=='Yes') | (df[bill_col] > current_date)]

    return new_sites, bts_out_uip, df_site_bts[[tw_index, bts_col, bill_col]]
    """Restruturar o script em CZ, DE, PT"""

def check_bts(df_tw, bts_tw_columns, tw_index, df_msa, bts_msa_column, msa_index):
    #Nested Function to make conditional validations
    def cond_bts_check(bts_msa, tw_bts_sites):
        bts_out_tw=[]
        if sorted(bts_msa) != sorted(tw_bts_sites):
            for i in tw_bts_sites:
                if i not in bts_msa:
                    bts_out_tw.append(i)

        return bts_out_tw

    bts_msa = msa[msa[bts_msa_column]=='Yes']
    bts_msa = [str(i) for i in bts_msa[msa_index]]

    tw_bts_sites = df_tw[df_tw[bts_tw_columns]=='Yes']
    tw_bts_sites = [str(i) for i in tw_bts_sites[tw_index]]

    #return of datas
    bts_out_tw = cond_bts_check(bts_msa, tw_bts_sites)
    df = pd.DataFrame(bts_out_tw, columns=['New Sites'])
    return df

def check_wip(df_tw,tw_index, wip_tw, tw_bts, df_msa, msa_index, wip_msa_col):

    #Nested Function to make conditional validations
    def cond_wip_check(wip_msa, tw_wip_sites):
        count_wip = 0
        wip_out_tw=[]
        if sorted(wip_msa) != sorted(tw_wip_sites):
            for i in tw_wip_sites:
                if i not in wip_msa:
                    count_wip += 1
                    wip_out_tw.append(i)

        return wip_out_tw

    wip_msa = df_msa[df_msa[wip_msa_col]=='Yes']
    wip_msa = [str(i) for i in wip_msa[msa_index]]

    tw_wip_sites = df_tw[df_tw[wip_tw]=='Yes']
    tw_wip_sites = [str(i) for i in tw_wip_sites[tw_index]]

    tw_wip_site_bts_flagged = df_tw[(df_tw[wip_tw]=='Yes')&(df_tw[tw_bts]=='Yes')]
    tw_wip_site_bts_flagged = tw_wip_site_bts_flagged[[tw_index, wip_tw, tw_bts]]

    wip_out_tw_list = cond_wip_check(wip_msa, tw_wip_sites)
    return wip_out_tw_list, tw_wip_site_bts_flagged
    """Reestrurar os script em PT, DE, CZ"""
    # Falta os outros países

def check_decommissioned(df,df_index, decom_col, doer_col):
    #c = country.lower()
    filtered = df[(df[decom_col]=='Yes')&(df[doer_col]=="")]
    return filtered[[df_index, decom_col, doer_col]]
    """Ajustar para CZ, DE, PT"""
    
def check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col):
    
    t = type_col.lower()
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    if not df_tw[date_col].empty:
        if t == 'doer':
            filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col].astype('datetime64[ns]') < current_date)]
            return filtered[[tw_index, status_col, date_col]] 
        else:
            filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col].astype('datetime64[ns]').empty)]
            return filtered[[tw_index, status_col, date_col]] 
    else:
        print('Nothing wrong in services sites!')
    """Ajustar para CZ, DE, PT, IE"""

def check_tw_doer_planned(df_tw, tw_index, doer_col, status_col):
    """Only GR until now"""
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    if not df_tw[doer_col].empty:
        filtered = df_tw[(df_tw[status_col]=='Planned')&(not df_tw[doer_col].astype('datetime64[ns]').empty)]
        return filtered[[tw_index, status_col, doer_col]]  
    else:
        print('Nothing wrong in services sites!')
                                                              
def check_mom_bts(df_tw, tw_index, tw_col, df_msa,msa_index, msa_col):

    #c = country   
    msa_bts = df_msa[df_msa[msa_col]=='Yes']
    msa_bts_sites = [i for i in msa_bts[msa_index]]

    tw_bts = df_tw[df_tw[tw_col]=='Yes']
    tw_bts_sites = [i for i in tw_bts[tw_index]]

    out_tower_bts = [i for i in msa_bts_sites if i not in tw_bts_sites]
    filtered = tw_bts[tw_bts[tw_index].isin(out_tower_bts)]
    return filtered[[tw_index, tw_col]]         

def check_lc_ta_dates(df,tw_index, start_date,end_date):

        filtered = df[df[start_date] > df[end_date]]
        return filtered[[tw_index, start_date,end_date]]

def check_uip_tw(df_tw,tw_index, status_tw_col, decom_col, tw_bts_col, tw_critical_col, df_uip, uip_sites, country):
    t1 = ['pt', 'de', 'cz', 'ie', 'es', 'ro', 'hu']
    if country.lower() in t1:
        #tw_active = df_tw[df_tw['Site Status']=='In Service']
        count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

        # check number of sites that are in uip file and doesn't have in df_tw
        in_service_uip_sites = []
        if not set(count_tw_sites).intersection(uip_sites):
            in_service_uip_sites = [i for i in uip_sites if i not in count_tw_sites]
        in_service_uip_sites = pd.DataFrame(in_service_uip_sites,columns=['Site In service out of UIP File!'])
        
        #check for decomissioned site not in uip files
        if decom_col != "":
            tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index] if i in uip_sites]
            decomiss_sites_in_uip = pd.DataFrame(tw_decomiss, columns=['Decomissioned Site in UIP File'])
        
            #Check BTS sites
            bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
            bts_sites_out_uip = []
            if not set(bts_sites).intersection(uip_sites):
                bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
            bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIP File'])

            #  Check for UIP critical sites 
            uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                        df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
            bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
            critical = []
            if len(uip_critical) > 0:
                if set(uip_critical).intersection(bts_tw_critical):
                    critical = [i for i in bts_tw_critical if i not in uip_critical]
            critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIP File'])

        
            return in_service_uip_sites, decomiss_sites_in_uip, bts_sites_out_uip, critical
        else:
            #Check BTS sites
            bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
            bts_sites_out_uip = []
            if not set(bts_sites).intersection(uip_sites):
                bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
            bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIP File'])

            #  Check for UIP critical sites 
            uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                        df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
            bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
            critical = []
            if len(uip_critical) > 0:
                if set(uip_critical).intersection(bts_tw_critical):
                    critical = [i for i in bts_tw_critical if i not in uip_critical]
            critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIP File'])
            return in_service_uip_sites, bts_sites_out_uip, critical
    else:
        #tw_active = df_tw[df_tw['Site Status']=='In Service']
        count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

        # check number of sites that are in uip file and doesn't have in df_tw
        in_service_uip_sites = []
        if not set(count_tw_sites).intersection(uip_sites):
            in_service_uip_sites = [i for i in uip_sites if i not in count_tw_sites]
        in_service_uip_sites = pd.DataFrame(in_service_uip_sites,columns=['Site In service out of UIP File!'])
        
        #check for decomissioned site not in uip files
        tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index]]
        decomiss_sites_in_uip = []
        if set(tw_decomiss).intersection(uip_sites):
            decomiss_sites_in_uip = [i for i in tw_decomiss if i in uip_sites]
        decomiss_sites_in_uip = pd.DataFrame(decomiss_sites_in_uip, columns=['Decomissioned Site in UIP File'])
        
        #Check BTS sites
        bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
        bts_sites_out_uip = []
        if not set(bts_sites).intersection(uip_sites):
            bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
        bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIP File'])

        #  Check for UIP critical sites 
        uip = [i for i in df_uip['Site_ID']]
        bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Yes'][tw_index]
        critical = []
        if set(uip).intersection(bts_tw_critical):
            critical = [i for i in bts_tw_critical if i not in uip]
        critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% out in UIP File'])

        return in_service_uip_sites, decomiss_sites_in_uip, bts_sites_out_uip, critical

def check_commercial(path_current, path_before, col_replace, col_names, merge_cols, col_order):
    
    def check_uip_commercial_values(df_actual, df_before, merge_cols):
        df_atual = uip_comercial_actual
        df_ant = uip_comercial_before
        df_cross = pd.merge(df_actual, df_before, on=merge_cols, \
                            how='left', suffixes=('_actual', '_before'), indicator='Exist')
        df_cross['Exist'] = np.where(df_cross.Exist == 'both', 'Yes', 'No')
        df_cross['Equal Values'] = df_cross['Exist']
        
        return df_cross
    # Check for commercial Values into current UIP File and compare with UIP File before
    uip_comercial_actual = pd.read_excel(path_current,sheet_name='Commercial', names=col_names)
    #uip_comercial_actual = uip_comercial_actual[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_actual = replace_values(uip_comercial_actual, col_replace, value=0)

    uip_comercial_before = pd.read_excel(path_before,sheet_name='Commercial', names=col_names )
    #uip_comercial_before = uip_comercial_before[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_before = replace_values(uip_comercial_before, col_replace, value=0)

    df_commercial =  check_uip_commercial_values(uip_comercial_actual, uip_comercial_before, merge_cols)

    df_commercial = df_commercial.reindex(columns=col_order)
    df_commercial_diffs = df_commercial[df_commercial['Equal Values']=='No']
    return df_commercial_diffs

def general_log_erros(df_list, sheet_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')   
    for dataframe, sheet in zip(df_list, sheet_list):
        dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0)   
    writer.save() 

In [ ]:
path_msa = '/content/TowerDB_Hungary_20210630.csv'
msa = pd.read_csv(path_msa, encoding='latin')

"""
msa_cols = lower_str(list(msa.columns))
msa.columns = msa_cols

path_tw = '/content/TowerDB_Hungary_20210731.csv'
towerdb = pd.read_csv(path_tw, encoding='latin')"""
list(msa.columns)

In [ ]:
path_msa = '/content/TowerDB_Hungary_20210630.csv'
msa = pd.read_csv(path_msa, encoding='latin')

msa_cols = lower_str(list(msa.columns))
msa.columns = msa_cols

path_tw = '/content/TowerDB_Hungary_20210731.csv'
towerdb = pd.read_csv(path_tw, encoding='latin')
towerdb.columns = msa_cols

"""Check Columns Received"""             
df_cols = check_columns_received(towerdb, msa_cols)
df_cols
#No columns missing

,Column(s) Missing


In [ ]:
"""Defining variables which is gonna be reusable in checks"""
tw_index = 'code'
tw_doer = 'date of equipment removal'
tw_status = 'active / not active'
tw_bts = 'bts_site'
tw_bill = 'ready for active installation ("rfai") date'
tw_wip = 'wip_site'
#tw_decom = 'Decommissioned sites'
tw_critical = 'critical site'


msa_index ='code'
msa_doer = 'date of equipment removal'
msa_status = 'active / not active'
msa_bts = 'bts_site'
msa_bill = 'ready for active installation ("rfai") date'
msa_wip = 'wip_site'
#msa_decom = 'Decommissioned sites'
msa_critical = 'critical site'

First Check - Dates Formats (dd/mm/YYYY)

Columns: Date of equipment removal

In [ ]:
"""You need to convert all values in cols for string format to check"""
# Columns to functions
dates_doer = [tw_index, tw_doer]
#Columns to parser
doer=[tw_doer]

#actives_1 = towerdb[towerdb[tw_status]=='In Service']
no_actives_1 = towerdb[~(towerdb[tw_status]=='In Service')]

#date_parser(towerdb, bill, "%d/%m/%Y", 'no')
date_parser(no_actives_1, doer, "%d/%m/%Y", '')


#Checking columns for errors
# actives_dates_errors = check_date_columns(actives_1, tw_index, dates_bill, 2) 
# Actives sites with blank billing trigger date
no_actives_dates_errors = check_date_columns(no_actives_1, tw_index, dates_doer, 2) 
no_actives_dates_errors

No one columns with incorrect date format!


Second Check - TW Amount value General (xxx.xx)

Column(s): ???

In [ ]:
amount_cols = [tw_index, """???"""]
df_amount_errors = check_amounts(actives, tw_index, amount_cols, 1)
#No one error

Thirth - Check Picklist values All sites

Do this check in all sites

Check the picklist for each case

Check - Remove "N/A", "0" or "-" values

In [ ]:
bill_cols = ['confirmed skylon scope','code','categorization by transmission sys', 'categorization by site type',\
             'additional information  room configuration (including container information)', 'type of air cooling', \
             'active sharing arrangements involving the operator', 'critical site','strategic site','telenor tenant','telekom tenant',\
             'power supply for tenants','energy category', 'ready for active installation ("rfai") date', 'wip_site','bts_site','strategic_site_bucket',\
             'critical_site_beyond_10','subsequent_sharing_arrangement','first_active_sharing_start_date','first_active_sharing_end_date',\
             'active sharing deployment types','sites_as_metered_estimated']
towerdb = replace_values(towerdb, bill_cols)

In [ ]:
towerdb[[tw_bill]].value_counts()

Thirth - Check Picklist values All sites

Do this check in all sites

Check the picklist for each case

In [ ]:
picklist_tw_general = {
    'categorization by transmission sys': ['Macro', 'Macro+DAS', 'Public DAS','Transmission']
}
pick_col_general = ['code', 'categorization by transmission sys']

df_general_pick = check_picklist(towerdb, tw_index, pick_col_general, picklist_tw_general)
df_general_pick

Check Picklist and dates formats for In service sites



In [ ]:
picklis_dict = {
    'categorization by transmission sys': ['Macro', 'Macro+DAS', 'Public DAS','Transmission'],
    'categorization by site type': ['DAS passive','GBT','RTT', 'RTT+DAS passive'],
    'type of air cooling' : ['Yes', 'No'],
    'critical site': ['Yes', 'No'],
    'strategic site': ['Yes', 'No'],
    'power supply for tenants': ['AC', 'DC', 'no power'],
    'bts_site': ['Yes', 'No'],
    'strategic site': ['Yes', 'No'],
    'critical site': ['Yes', 'No'],
    'strategic_site_bucket': ['Non Strategic'],
    'critical_site_beyond_10':['Within 10%','Non Critical'],
    'wip_site': ['Yes', 'No'],
    'sites_as_metered_estimated': ['Estimated Model','Metered Model']
}

picklist_cols = ['code','categorization by transmission sys', 'categorization by site type', 'type of air cooling',\
                 'critical site','strategic site','power supply for tenants', 'wip_site','bts_site',\
                 'strategic_site_bucket','critical_site_beyond_10','sites_as_metered_estimated','strategic site','critical site']

date_cols = ['code','infrastructure ready (existing)/ to be ready (new)', 'ready for active installation ("rfai") date', 'first_active_sharing_start_date','first_active_sharing_end_date']
actives = towerdb[towerdb[tw_status]=='In Service']



In [ ]:
actives = replace_values(actives,picklist_cols, '')
df_in_service_picklist = check_picklist(actives, tw_index, picklist_cols, picklis_dict)
df_in_service_picklist
#muitos errors

In [ ]:
start_dates = ['infrastructure ready (existing)/ to be ready (new)', 'ready for active installation ("rfai") date', 'first_active_sharing_start_date','first_active_sharing_end_date']

#date_parser(actives, start_dates, "%d/%m/%Y", 'no')
#actives['First_Active_Sharing_End_Date'] = actives['First_Active_Sharing_End_Date'].fillna('')
df_in_service_dates = check_date_columns(actives, tw_index, date_cols, 2)
df_in_service_dates
#Muitos Campos em branco

Fifth Check BTS Flagged(Billing Trigger and Commercial)

In [ ]:
status = 'Yes'
df_bts_flagged = check_tw_bill_doer(actives, tw_index,tw_bill, tw_bts, status, 'bill')
df_bts_flagged
#No errors

Check MoM Sites (BTS)

In [ ]:
""" BTS sites"""
path_uip = '/content/UserInput_Hungary_20210731.xlsx'
uip_names = ['Site_ID','BTS site applicable charge (Annual)',\
             'Commercials for sites beyond 10% cap of critical sites (Annual)']
uip = pd.read_excel(path_uip ,sheet_name='SiteLevel',usecols=[0,1,2],skiprows=2)
uip.columns = uip_names

msa_sites = [str(i) for i in msa[msa_index]]
tw_sites = [str(i) for i in towerdb[tw_index]]
uip_sites = [str(i) for i in uip['Site_ID']]

In [ ]:
df_mom_bts = check_mom_bts(actives, tw_index, tw_bts, msa, msa_index, msa_bts)
df_mom_bts
# No one error df_mom_bts

Check UIP sites with towerDB
Some countries doesn't have decommissioned column

In [ ]:
in_service_uip_sites, bts_sites_out_uip, critical = check_uip_tw(towerdb,tw_index, tw_status, '', tw_bts,tw_critical,\
                                                                 uip, uip_sites, 'hu')
#no errors

Commercial

In [ ]:
path_before = '/content/UserInput_Hungary_20210630.xlsx'
names = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2', 'Data_Type', 'Input_Value',\
        'Description/Instruction', 'Frequency of Update']
merge_cols = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2', 'Data_Type', \
              'Description/Instruction', 'Frequency of Update']
cols_ordered = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2','Data_Type','Input_Value_actual',\
                'Input_Value_before','Equal Values','Description/Instruction','Frequency of Update']
col_replaced = ['Input_Value']
df_commercial_diffs = check_commercial(path_uip, path_before,col_replaced, names, merge_cols, cols_ordered)
#df_commercial_diffs = check_commercial(path_uip, path_before, replace_values, names, merge_cols, cols_ordered)
df_commercial_diffs

,Charge_Type,Sub_Charge_Type,Param1,Param2,Data_Type,Input_Value_actual,Input_Value_before,Equal Values,Description/Instruction,Frequency of Update
9,MNO & BARTER TOP-UP ARRANGEMENTS,MNO TOP-UP ARRANGEMENTS,TOP-UP THRESHOLD,0,NUMBER,1699500,NaN,No,- Annual value (in HUF) as per the MSA\n- To b...,Only in case of change in MSA/Side letter
47,MNO & BARTER TOP-UP ARRANGEMENTS,BARTER TENANCY ARRANGEMENTS,TOP-UP FEE,DIGI,NUMBER,1699500,NaN,No,- Annual value (in HUF) as per the MSA\n- To b...,Only in case of change in MSA/Side letter
48,MNO & BARTER TOP-UP ARRANGEMENTS,BARTER TENANCY ARRANGEMENTS,TOP-UP FEE,OTMO,NUMBER,412000,NaN,No,- Annual value (in HUF) as per the MSA\n- To b...,Only in case of change in MSA/Side letter


Excel Log

In [ ]:
df_list = [df_in_service_dates, df_in_service_picklist]
sheetnames = ['In services dates errrors', 'In servive picklist errors']

path = '/content/towerdb_hu_errors.xlsx'
general_log_erros(df_list, sheetnames, path)

TA Input

In [ ]:

ta.columns

Index(['SiteID', 'SiteID.1', 'In/out of scope', 'Tenant Agreement ID',
       'Agreement relates to DAS site?', 'Tenant (name/ID)', 'Classification',
       'MNO classification', 'Base fee (@ signing) in HUF',
       'Additional fee (@ signing) in HUF',
       'Annual Fee per Tenant (@ signing) in HUF',
       'Annual Energy Fee  (@ signing)', 'Annual Maintenance Fee  (@ signing)',
       'Other Services Fee  (@ signing)', 'Base fee in HUF',
       'Additional fee in HUF', 'Current Annual Fee per Tenant in HUF',
       'Current Annual Energy Fee in HUF', 'Starting date',
       'Expiring date (raw data)', 'Expiring date (in practice)',
       'Renewal Option', 'Expiring date after renewal', 'Terms of payments',
       'Payment type', 'Index', 'Percentage', 'Indexation driver',
       'VAT Suject', 'Percentage (VAT)', 'Termination Date',
       'Residual duration'],
      dtype='object')

In [ ]:
#ta[ta['SiteID']=='5816']['Base fee (@ signing) in HUF']
ta[['Base fee (@ signing) in HUF']].value_counts()

In [ ]:
pathta = '/content/TA_Input_Hungary_20210731.csv'

ta = pd.read_csv(pathta,  encoding='latin')

ta_leases = ['Base fee (@ signing) in HUF']
ta = replace_values(ta,ta_leases, '')

ta_cols = ['SiteID', 'Base fee (@ signing) in HUF']  
df_ta_amount = check_amounts(ta, 'SiteID', ta_cols)
print(df_ta_amount)
# No errors

df_ta_dates = check_lc_ta_dates(ta,'SiteID', 'Starting date', 'Expiring date (raw data)')
#No erros

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


     SiteID                 Base fee (@ signing) in HUF
1075   1075  Incorret Format to separate decimal values
1077   1077  Incorret Format to separate decimal values
1107   1107  Incorret Format to separate decimal values
1137   1137  Incorret Format to separate decimal values
1139   1139  Incorret Format to separate decimal values
...     ...                                         ...
4770   4770  Incorret Format to separate decimal values
5429   5429  Incorret Format to separate decimal values
5513   5513  Incorret Format to separate decimal values
5516   5516  Incorret Format to separate decimal values
5816   5816  Incorret Format to separate decimal values

[635 rows x 2 columns]


LC Input

In [ ]:
lc[['Current annual lease fees in HUF']].value_counts()

In [ ]:
 #separador virgulas
pathlc = '/content/LC_Input_Hungary_20210731.csv'

lc = pd.read_csv(pathlc,  encoding='latin')
leases = ['Current annual lease fees in HUF']
lc = replace_values(lc,leases, '')

lc_cols = ['Code', 'Current annual lease fees in HUF']  

df_lc_amount = check_amounts(lc, 'Code', lc_cols)
# No errors

df_lc_dates = check_lc_ta_dates(lc,'Code', 'Starting date', 'Expiring date (raw data)')
#No erros

In [ ]:
p = re.compile(r'^\d+(\.\d*[1-9])?$')
v = '10200,01'
if not str(v).__contains__(','):
#if p.match(v) == None:
    print("Erro")
else:
    print(v)

10200,01
